In [78]:
import os, re
import pandas as pd
import numpy as np

# #显示所有列
# pd.set_option('display.max_columns', None)

# #显示所有行
# pd.set_option('display.max_rows', None)

pd.set_option('max_colwidth',100) 

In [79]:
folders = [
    # r'D:\code\forecast_model\notebook\20250811_base',
    # r'D:\code\forecast_model\notebook\20250811_small_large',
    # r'D:\code\forecast_model\notebook\20250811_extraLarge',
    # r"D:\code\forecast_model\notebook\20250815_base_large",
    # r'D:\code\forecast_model\notebook\20250817_base',
    # r'D:\code\forecast_model\notebook\20250820_base',
    # r'D:\code\forecast_model\notebook\20250820_bigger',
    # r'D:\code\forecast_model\notebook\20250821_mlp_base2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline3',
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline2",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline3",
    # r'D:\code\forecast_model\notebook\20250825_tcn_baseline',
    # r"D:\code\forecast_model\notebook\20250826_tcn_filt_days",
    r'D:\code\forecast_model\notebook\20250828_tcn_data',
    r'D:\code\forecast_model\notebook\20250829_tcn_data',
    r"D:\code\forecast_model\notebook\20250829_tcn_data2",
    ]

In [80]:
data = pd.DataFrame()
for folder in folders:
    for i in os.listdir(folder):
        if 'muilt_model' == i:continue
        # 20250811_base_P100_DeepLOB_v2_input_indepent	0.558035	2.77h
        # 20250811_base_P100_DeepLOB_v2_overfit
        if 'input_indepent' in i or 'overfit' in i: continue

        if i.endswith('.csv'): continue
        if i.endswith('.xlsx'): continue

        code = i.split('_')[3]
        fold = i.split('_')[4]

        train_folder = os.path.join(folder, i)
        if not os.path.isdir(train_folder): continue

        result_file = os.path.join(train_folder, 'result.csv')
        if not os.path.exists(result_file): continue

        # 修复替换
        _data = open(result_file, 'r').readlines()
        _data[1] = _data[1].replace('(30, 2, 1)', '30@2@1')
        with open(result_file, 'w') as f:
            f.writelines(_data)

        _data = pd.read_csv(result_file)
        # _data['describe.1'] = _data['describe.1'] + "_" + code + f"_fold{fold}"

        # 判断是否有nan
        if _data.isnull().values.any(): 
            print(i)
            continue

        if 'test_class_f1_0' in list(_data):
            _data[f'test_mean_class_f1'] = (_data[f'test_class_f1_0'] + _data[f'test_class_f1_1']) / 2
        elif 'test_final_class_f1_0' in list(_data):
            # 同时采用 best/final
            d2 = _data.copy()
            d2[f'test_mean_class_f1'] = _data[f'test_best_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                d2[f'test_mean_class_f1'] += _data[f'test_best_class_f1_{i}']
            d2[f'test_mean_class_f1'] = d2[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            d2['train_title'] += '_best'

            _data[f'test_mean_class_f1'] = _data[f'test_final_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                _data[f'test_mean_class_f1'] += _data[f'test_final_class_f1_{i}']
            _data[f'test_mean_class_f1'] = _data[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            _data['train_title'] += '_final'

            _data = pd.concat([_data, d2], ignore_index=True)

        # for _type in ['train', 'val', 'test_best', 'test_final', 'test_dummy']:
        #     _data[f'{_type}_mean_class_f1'] = (_data[f'{_type}_class_f1_0'] + _data[f'{_type}_class_f1_1']) / 2

        # for _type in ['train', 'val', 'test_best', 'test_final']:
            # _data[f'{_type}_mean_class_f1_enhanced_pct'] = 100 * (_data[f'{_type}_mean_class_f1'] - _data['test_dummy_mean_class_f1']) / _data['test_dummy_mean_class_f1']

        data = pd.concat([data, _data], ignore_index=True)

try:
    data['mean_test_01_f1'] = (data['test_best_class_f1_0'] + data['test_best_class_f1_1']) / 2
    data['mean_val_01_f1'] = (data['val_class_f1_0'] + data['val_class_f1_1']) / 2
except:
    pass

# 只使用 top 数据集 predict_n100
# data = data.loc[data['train_title'].str.contains('top5')].reset_index(drop=True)
# data = data.loc[data['train_title'].str.contains('predict_n100')].reset_index(drop=True)

data

,train_title,root,alist_upload_folder,amp,epochs,batch_n,batch_size,learning_rate,abs_learning_rate,no_better_stop,...,test_dummy_mean_class_f1,train_mean_class_f1_enhanced_pct,val_mean_class_f1_enhanced_pct,test_best_mean_class_f1_enhanced_pct,test_final_mean_class_f1_enhanced_pct,each_epoch_cost,cost,test_mean_class_f1,mean_test_01_f1,mean_val_01_f1
0,20250828_data_P100_bc_420_seed0_IDX0_final,20250828_data_P100_bc_420_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.499272,99.253665,40.872588,55.632824,48.863974,0.03h,5.97h,0.741045,0.777031,0.703338
1,20250828_data_P100_bc_420_seed0_IDX0_best,20250828_data_P100_bc_420_seed0_IDX0,train_data,no,200,128,8192,0.0384,0,0,...,0.499272,99.253665,40.872588,55.632824,48.863974,0.03h,5.97h,0.747575,0.777031,0.703338
2,20250828_data_P100_bc_420_seed1_IDX2_final,20250828_data_P100_bc_420_seed1_IDX2,train_data,no,200,128,8192,0.0384,0,0,...,0.501937,98.172216,38.031681,43.999898,44.873419,0.03h,6.31h,0.728759,0.722788,0.692832
3,20250828_data_P100_bc_420_seed1_IDX2_best,20250828_data_P100_bc_420_seed1_IDX2,train_data,no,200,128,8192,0.0384,0,0,...,0.501937,98.172216,38.031681,43.999898,44.873419,0.03h,6.31h,0.748459,0.722788,0.692832
4,20250828_data_P100_bc_420_seed2_IDX4_final,20250828_data_P100_bc_420_seed2_IDX4,train_data,no,200,128,8192,0.0384,0,0,...,0.500070,98.935943,36.605217,41.749954,44.407890,0.03h,6.04h,0.729337,0.708849,0.683121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,20250829_data_bc_P100_bc_top5_420_seed3_IDX23_best,20250829_data_bc_P100_bc_top5_420_seed3_IDX23,train_data,no,180,128,8192,0.0384,0,0,...,0.499523,88.659994,30.482179,40.034312,26.427632,0.06h,10.67h,0.707925,0.699504,0.651788
153,20250829_data_bc_P100_bc_top5_420_seed4_IDX29_final,20250829_data_bc_P100_bc_top5_420_seed4_IDX29,train_data,no,180,128,8192,0.0384,0,0,...,0.500299,88.082262,32.549250,44.327490,28.575364,0.06h,10.98h,0.631885,0.722069,0.663142
154,20250829_data_bc_P100_bc_top5_420_seed4_IDX29_best,20250829_data_bc_P100_bc_top5_420_seed4_IDX29,train_data,no,180,128,8192,0.0384,0,0,...,0.500299,88.082262,32.549250,44.327490,28.575364,0.06h,10.98h,0.733057,0.722069,0.663142
155,20250829_data_P100_dl_top5_420_seed0_IDX5_final,20250829_data_P100_dl_top5_420_seed0_IDX5,train_data,no,180,128,8192,0.0384,0,0,...,0.499752,86.412601,34.203236,37.870934,31.543942,0.06h,10.40h,0.662872,0.689012,0.670683


In [81]:
if len(folders) == 1:
    data.to_csv(os.path.join(folders[0], 'result.csv'), index=False)


In [82]:
list(data)

['train_title',
 'root',
 'alist_upload_folder',
 'amp',
 'epochs',
 'batch_n',
 'batch_size',
 'learning_rate',
 'abs_learning_rate',
 'no_better_stop',
 'checkpointing_steps',
 'label_smoothing',
 'weight_decay',
 'classify',
 'y_n',
 'debug',
 'test',
 'seed',
 'label_train',
 'label_val',
 'label_test_final',
 'model',
 'grad_norm_best',
 'grad_norm',
 'total_grad_norm_best',
 'total_grad_norm',
 'train_loss_best',
 'train_loss',
 'train_acc_best',
 'train_acc',
 'train_f1_best',
 'train_f1',
 'train_recall_best',
 'train_recall',
 'train_class_acc_0_best',
 'train_class_acc_0',
 'train_class_mcc_0_best',
 'train_class_mcc_0',
 'train_class_f1_0_best',
 'train_class_f1_0',
 'train_class_recall_0_best',
 'train_class_recall_0',
 'train_class_acc_1_best',
 'train_class_acc_1',
 'train_class_mcc_1_best',
 'train_class_mcc_1',
 'train_class_f1_1_best',
 'train_class_f1_1',
 'train_class_recall_1_best',
 'train_class_recall_1',
 'val_loss_best',
 'val_loss',
 'val_acc_best',
 'val_acc',

In [83]:
data[['train_title', 'label_train']]

,train_title,label_train
0,20250828_data_P100_bc_420_seed0_IDX0_final,1(309275)@1(309275)
1,20250828_data_P100_bc_420_seed0_IDX0_best,1(309275)@1(309275)
2,20250828_data_P100_bc_420_seed1_IDX2_final,1(309275)@1(309275)
3,20250828_data_P100_bc_420_seed1_IDX2_best,1(309275)@1(309275)
4,20250828_data_P100_bc_420_seed2_IDX4_final,1(309275)@1(309275)
...,...,...
152,20250829_data_bc_P100_bc_top5_420_seed3_IDX23_best,1(980349)@1(980349)
153,20250829_data_bc_P100_bc_top5_420_seed4_IDX29_final,1(980349)@1(980349)
154,20250829_data_bc_P100_bc_top5_420_seed4_IDX29_best,1(980349)@1(980349)
155,20250829_data_P100_dl_top5_420_seed0_IDX5_final,1(955676)@1(955676)


In [89]:
# 多seed平均
key_col_name = ['train_f1']
key_col_name = ['train_loss', 'train_f1', 'val_f1', 'val_f1_best', 'val_loss', 'label_train']

d = data.loc[:, key_col_name + ['cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'seed' not in i]))
d['label_train'] = d['label_train'].apply(lambda x:int(x.split('@')[-1][2:-1])*2)

# 只考察 final
_match_bool = data['train_title'].apply(lambda x: 'final' in x or ('final' not in x and 'best' not in x))
d = d.loc[_match_bool]
# 去除 final/best 的标记
d['train_title'] = d['train_title'].apply(lambda x: x.replace('_final', '').replace('_best', ''))

# # 关键字过滤
# keys = ['bc_top5', 'TimeSeriesStaticModelx8_bc', 'P100_bc_420']
# _match_key_bool = data['train_title'].apply(lambda x: any([k in x for k in keys]))
# d = d.loc[_match_key_bool]

# # 根据 idx 区分类别
# # 用于未区分title的情况
# _cls = d['train_title'].apply(lambda x: [(int(i.replace('IDX', '')) % 3) for i in x.split('_') if 'IDX' in i][0])
# _cls = _cls.apply(lambda x: ['nomove30', 'nomove50', 'nomove70'][x])
# d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i])) + '_' + _cls

# title 已经区分
d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(round(x,2)) + 'h')
d.sort_values('val_f1_best',ascending=False)


,train_loss,train_f1,val_f1,val_f1_best,val_loss,label_train,cost
train_title,,,,,,,
20250829_data_bc_P100_bc_top5_300,0.044520,0.983322,0.703182,0.768434,2.310449,1021926.0,7.17h
20250829_data_P100_only15_420,0.022816,0.991716,0.693835,0.749024,2.810042,870722.0,6.33h
20250829_data_P100_dl_top5_300,0.043230,0.983880,0.689225,0.735812,2.389834,994196.0,6.97h
20250829_data_bc_P100_bc_top5_200,0.013743,0.995122,0.709904,0.733480,2.395001,524014.0,5.27h
20250829_data_bc_P100_bc_only15_420,0.023513,0.991422,0.690460,0.731607,2.803140,887654.0,6.52h
20250829_data_P100_dl_top5_200,0.014276,0.994986,0.718931,0.729681,2.348435,510076.0,5.13h
20250828_data_P100_deeplob_420,0.014125,0.994961,0.681926,0.728979,2.998034,603498.0,6.24h
20250829_data_bc_P100_bc_top5_420,0.165308,0.931016,0.652201,0.717970,1.557495,1960698.0,10.7h
20250828_data_P100_bc_420,0.015150,0.994624,0.695981,0.711016,2.884220,618550.0,6.41h


In [85]:
# 多IDX平均
key_col_name = 'val_f1'
key_col_name = 'label_train'

d = data.loc[:, [key_col_name, 'cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i and 'seed' not in i ]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values([key_col_name],ascending=False)

TypeError: agg function failed [how->mean,dtype->object]